In [1]:
import numpy as np
import pandas as pd
import reverse_geocoder as rg 
import pprint 
from datetime import datetime,time, date
import requests
import json

In [2]:
df = pd.read_csv('yellow_tripdata_2016-06.csv')

In [3]:
df.shape

(11135470, 19)

In [4]:
df.head()
# df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.0,0.3,27.30
2,2,2016-06-09 21:06:36,2016-06-09 21:13:10,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.0,0.3,28.30
4,2,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.0,0.3,17.76


In [5]:
df.tpep_pickup_datetime = df.tpep_pickup_datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df.tpep_dropoff_datetime = df.tpep_dropoff_datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df = df.sort_values(by = 'tpep_pickup_datetime')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
95250,2,2016-06-01 00:00:00,2016-06-01 00:00:00,1,2.59,-73.997505,40.725487,1,N,-73.997742,40.744919,2,18.0,0.0,0.5,0.00,0.0,0.3,18.80
95249,1,2016-06-01 00:00:00,2016-06-01 00:14:11,4,2.30,-73.972916,40.754993,1,N,-73.992264,40.725243,1,11.5,0.5,0.5,3.80,0.0,0.3,16.60
95248,1,2016-06-01 00:00:00,2016-06-01 00:15:25,1,5.90,-73.962227,40.760635,1,N,-73.922287,40.827213,1,19.5,0.5,0.5,4.16,0.0,0.3,24.96
95247,1,2016-06-01 00:00:00,2016-06-01 00:07:41,1,1.30,-74.008446,40.706024,1,N,-74.013390,40.709644,1,7.5,0.5,0.5,2.60,0.0,0.3,11.40
95253,2,2016-06-01 00:00:01,2016-06-01 00:03:00,1,0.76,-73.976189,40.775909,1,N,-73.982300,40.767860,1,4.5,0.5,0.5,2.00,0.0,0.3,7.80


In [6]:
df = df.drop(columns = ['VendorID', 'store_and_fwd_flag', 'fare_amount', 'tip_amount', 'tolls_amount', 'extra', 'mta_tax', 'improvement_surcharge' ])

In [7]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount
95250,2016-06-01 00:00:00,2016-06-01 00:00:00,1,2.59,-73.997505,40.725487,1,-73.997742,40.744919,2,18.80
95249,2016-06-01 00:00:00,2016-06-01 00:14:11,4,2.30,-73.972916,40.754993,1,-73.992264,40.725243,1,16.60
95248,2016-06-01 00:00:00,2016-06-01 00:15:25,1,5.90,-73.962227,40.760635,1,-73.922287,40.827213,1,24.96
95247,2016-06-01 00:00:00,2016-06-01 00:07:41,1,1.30,-74.008446,40.706024,1,-74.013390,40.709644,1,11.40
95253,2016-06-01 00:00:01,2016-06-01 00:03:00,1,0.76,-73.976189,40.775909,1,-73.982300,40.767860,1,7.80


In [8]:
df['dropoff_lat_long_tuple'] = list(zip(df.dropoff_latitude, df.dropoff_longitude))
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple
95250,2016-06-01 00:00:00,2016-06-01 00:00:00,1,2.59,-73.997505,40.725487,1,-73.997742,40.744919,2,18.80,"(40.7449188232, -73.9977416992)"
95249,2016-06-01 00:00:00,2016-06-01 00:14:11,4,2.30,-73.972916,40.754993,1,-73.992264,40.725243,1,16.60,"(40.7252426147, -73.9922637939)"
95248,2016-06-01 00:00:00,2016-06-01 00:15:25,1,5.90,-73.962227,40.760635,1,-73.922287,40.827213,1,24.96,"(40.8272132874, -73.9222869873)"
95247,2016-06-01 00:00:00,2016-06-01 00:07:41,1,1.30,-74.008446,40.706024,1,-74.013390,40.709644,1,11.40,"(40.7096443176, -74.0133895874)"
95253,2016-06-01 00:00:01,2016-06-01 00:03:00,1,0.76,-73.976189,40.775909,1,-73.982300,40.767860,1,7.80,"(40.7678604126, -73.9822998047)"


In [9]:
df = df[(df.RatecodeID == 2) & (df.trip_distance > 0.1) & (df.pickup_longitude != 0) & (df.pickup_latitude != 0) & (df.dropoff_longitude != 0) & (df.dropoff_latitude != 0)]    #source hub = JFK, RatecodeID = 2
df.head()
# df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple
95389,2016-06-01 00:00:11,2016-06-01 00:32:31,1,18.10,-73.789818,40.643749,2,-73.994164,40.746147,1,72.89,"(40.7461471558, -73.9941635132)"
95395,2016-06-01 00:00:13,2016-06-01 00:29:04,1,17.50,-73.786156,40.642078,2,-73.947021,40.810623,2,58.34,"(40.8106231689, -73.9470214844)"
95415,2016-06-01 00:00:20,2016-06-01 00:15:56,1,10.20,-73.862228,40.765160,2,-73.939835,40.843483,1,60.00,"(40.8434829712, -73.9398345947)"
95540,2016-06-01 00:00:24,2016-06-01 00:31:00,5,16.97,-73.789894,40.643764,2,-73.951637,40.759575,1,63.36,"(40.7595748901, -73.9516372681)"
95538,2016-06-01 00:00:24,2016-06-01 00:22:41,1,16.30,-73.789696,40.646622,2,-73.975792,40.744896,1,69.99,"(40.7448959351, -73.9757919312)"


In [10]:
df.shape

(242309, 12)

In [11]:
neg_fare_index = list(df[df.total_amount < 0].index)
pos_fare_index = []

for index in neg_fare_index:
    pickup_time = df.loc[index,'tpep_pickup_datetime']
    drop_time = df.loc[index, 'tpep_dropoff_datetime']
    pickup_long = df.loc[index, 'pickup_longitude']
    pickup_lat = df.loc[index, 'pickup_latitude']
    dropoff_long = df.loc[index, 'dropoff_longitude']
    dropoff_lat = df.loc[index, 'dropoff_latitude']
    total_amount =  -df.loc[index, 'total_amount']
    
    
    
    req_index = list(df[(df.tpep_pickup_datetime == pickup_time) & (df.tpep_dropoff_datetime == drop_time) &
             (df.pickup_longitude == pickup_long) & (df.dropoff_longitude == dropoff_long) &
             (df.pickup_latitude == pickup_lat) & (df.dropoff_latitude == dropoff_lat) &
             (df.total_amount == total_amount)].index)
    pos_fare_index.extend(req_index)
    
remove_index = pos_fare_index + neg_fare_index





In [12]:
df = df[np.logical_not(df.index.isin(remove_index))]
df.shape

(242125, 12)

In [13]:
geo = rg.RGeocoder(mode=2, verbose=True)
results = geo.query(list(df.loc[:,'dropoff_lat_long_tuple']))
df['dropoff_city'] = [odict['name'] for odict in results]
df = df[df['dropoff_city'] == 'Manhattan']
df.shape

Loading formatted geocoded file...


(62346, 13)

In [14]:
df = df[df['passenger_count'] < 3]
df.shape

(51607, 13)

In [15]:
df = df.reset_index(drop = True)

In [16]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city
0,2016-06-01 00:00:55,2016-06-01 00:31:25,1,19.20,-73.790199,40.646770,2,-73.966484,40.789272,1,66.34,"(40.7892723083, -73.9664840698)",Manhattan
1,2016-06-01 00:01:35,2016-06-01 00:34:08,1,18.52,-73.789909,40.646751,2,-73.962631,40.801632,1,58.34,"(40.8016319275, -73.9626312256)",Manhattan
2,2016-06-01 00:01:56,2016-06-01 00:34:33,1,21.37,-73.782372,40.644585,2,-73.979279,40.776905,1,70.01,"(40.7769050598, -73.9792785645)",Manhattan
3,2016-06-01 00:02:36,2016-06-01 00:33:02,1,16.91,-73.789429,40.646759,2,-73.979111,40.766800,1,63.36,"(40.7667999268, -73.9791107178)",Manhattan
4,2016-06-01 00:03:54,2016-06-01 00:31:41,1,17.40,-73.789833,40.646851,2,-73.971542,40.762051,1,72.96,"(40.7620506287, -73.9715423584)",Manhattan


In [ ]:
df.